In [1]:
import os

In [2]:
os.chdir("..")

In [3]:
os.getcwd()

'/home/markus/Documents/Projects/conservation_synthesis_github'

In [4]:
from sources.databases.internal_databases import SQLiteDB
from sources.frontend.app import get_journals
from sources.databases.journal_name_issn_database import JournalNameIssnDatabase

In [5]:
import pandas as pd

In [6]:
db = SQLiteDB()
db.initialise()
data = pd.DataFrame(db._get_all_data(0, 2000000))
db.terminate()

In [7]:
import re
import datetime

standard_date = re.compile(r"(\d{4})-(\d{1,2})-(\d{1,2})")
partial_date = re.compile(r"(\d{4})-(\d{1,2})")
dot_date = re.compile(r"(\d{4}).0")

def fix_date(date):
    if date is None:
        return None
    
    match =  standard_date.fullmatch(date.strip())
    if match is not None:
        return datetime.date(int(match.group(1)), int(match.group(2)), int(match.group(3))).isoformat()
    
    match = dot_date.fullmatch(date.strip())
    if match is not None:
        return datetime.date(int(match.group(1)), 1, 1).isoformat()
    
    match = partial_date.fullmatch(date.strip())
    if match is not None:
        return datetime.date(int(match.group(1)), 1, 1).isoformat()
    
    raise Exception("OH NO!")
    
def is_date(date):
    if date is None :
        return False
    val = standard_date.fullmatch(date.strip()) is not None 
    val = val or partial_date.fullmatch(date.strip()) is not None
    val = val or dot_date.fullmatch(date.strip()) is not None
    if val is False:
        print(date)
    return val

In [ ]:
test = "2003.0"
fix_date(test)

In [ ]:
standard_date = re.compile(r"\d{4}.0")
standard_date.match()

In [ ]:
def get_first(ls):
    return list(map(lambda p: p[0], filter(lambda p: not p[1], ls)))

In [ ]:
return_val= data.apply(lambda row: (row[0], is_date(row[4])), axis=1)
data.set_index(keys=0, drop=True, inplace=True)
data.loc[get_first(return_val),:]

We can now fix stuff!

In [8]:
data.set_index(keys=0, drop=True, inplace=True)

In [13]:
data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,,,,,,,,,,,,,,,,,,
1,Test,abc;bed,1234,2021-03-07,,testrepo,EN,None,None,None,None,None,None,None,b'0',None,0,NaN
118,In Memoriam Charles Henry Peterson,Hunter S.Lenihan;JonathanGrabowski;FiorenzaMic...,10.3354/meps13649,2021-2-4,None,crossref,EN,Inter-Research Science Center,None,None,None,0171-8630,https://doi.org/10.3354/meps13649,2021-03-07,b'0',NA,0,NaN
119,Diverse herbaceous cover crops promote vineyar...,Michael J. Samways;René Gaigher;James S. Pryke...,10.1016/j.agee.2020.107222,2021-2,Abstract Agricultural intensification threaten...,openaire,EN,Elsevier BV,"Agriculture, Ecosystems & Environment",307,None,0171-8630,https://doi.org/10.1016/j.agee.2020.107222,2021-03-07,b'0',NA,1,0.981580
120,Sugar feeding by parasitoids inside and around...,Miriam Kishinevsky;Tamar Keasar,10.1016/j.agee.2020.107229,2021-2,Abstract Feeding on sugar-rich resources drama...,openaire,EN,Elsevier BV,"Agriculture, Ecosystems & Environment",307,None,0171-8630,https://doi.org/10.1016/j.agee.2020.107229,2021-03-07,b'0',NA,1,0.580991
121,Environmental soil quality and vegetable safet...,Chao Ma;Yusef Kianpoor Kalkhajeh;Wenyou Hu;Bia...,10.1016/j.agee.2020.107230,2021-2,Abstract Rapid expansion of greenhouse vegetab...,openaire,EN,Elsevier BV,"Agriculture, Ecosystems & Environment",307,None,0171-8630,https://doi.org/10.1016/j.agee.2020.107230,2021-03-07,b'0',NA,0,0.052421


In [17]:
col_name = "publication_date"

In [19]:
db = SQLiteDB()
db.initialise()
cnt = 0
for ind, row in data.iterrows():
    db._update_col(ind, col_name, fix_date(row[4]))
    cnt += 1
db.terminate()

## Fix Journal Names

In [145]:
names = pd.unique(data.loc[:,9])

In [146]:
names = list(names)

In [147]:
names = set(names) - {None, ''}

In [148]:
with JournalNameIssnDatabase() as db:
    name_issn = db._name_issn_db

In [149]:
journals = name_issn.keys()

In [150]:
from collections import defaultdict
mapping = defaultdict(set)
for name in names:
    if name.lower() in journals:
        mapping[name.lower()].add(name.lower())
for name in journals:
    mapping[name.lower()].add(name.lower())

mapping = defaultdict(set, {name: set(names) for name, names in mapping.items()})

In [151]:
import itertools

In [152]:
remap = {"anzeiger fur schaedlingskunde" : ['Anzeiger fur Sch<html_ent glyph="@auml;" ascii="a"/>dlingskunde',
 'Anzeiger für Schädlingskunde',
 'Anzeiger für Schädlingskunde = Journal of pest science',
 'Anzeiger f�r Sch�dlingskunde'],
"acta agriculturae Scandinavica, section b - plant soil science" : ['Acta Agriculturae Scandinavica, Section B - Soil & Plant Science',
 'Acta Agriculturae Scandinavica, Section B — Soil & Plant Science'],
'Acta Agrobotanica' : ['Acta Agrobotanica'],
 'Plant Ecology & Diversity' : [', Plant Ecology & Diversity'],
 'current biology' : [
 'Curr Biol',
 'Curr Biol.',
 'Curr Biolo',
 'Current Biol.',
 'Current Biology Volume 23, Issue 12, Pages 1120\\u20131125',
 'Current Issues with Influenza Management',
 'Current biology : CB'],
 'ecosystems': [ 'Ecosystems (New York, N.Y.)',
 'Ecosystems (New York, N.y.)',
 'Ecosystems, 15(3)2012'],
 "Flora - Morphology, Distribution, Functional Ecology of Plants":[ 'Flora',
 'Flora.', 'Flora: Morphology, Distribution, Functional Ecology of Plants']
}

In [153]:
for key, entries in remap.items():
    if key is not None:
        mapping[key.lower()] |= set([s.lower() for s in entries])
    else:
        mapping[None] |= set([s.lower() for s in entries])

In [154]:
mapping["Limnologica - Ecology and Management of Inland Waters".lower()] |= {"limnologica"}

In [155]:
remaining_names = {n.lower() for n in names} - set(itertools.chain.from_iterable(mapping.values()))

In [156]:
for name in remaining_names:
    name = name.lower().strip()
    
    if name.startswith("the"):
        wtname = name[3:].strip()
        if wtname in mapping:
            mapping[wtname] |= {name}
        else:
            mapping[name] |= {name}
    else:
        mapping[name] |= {name}

In [157]:
remaining_names = {n.lower() for n in names} - set(itertools.chain.from_iterable(mapping.values()))

In [158]:
mapping[None] = {None, ""}

In [159]:
name_to_standard = {}

for s_name, set_names in mapping.items():
    for sn in set_names:
        name_to_standard[sn] = s_name

In [138]:
name_to_standard

{'aquatic conservation: marine and freshwater ecosystems': 'aquatic conservation: marine and freshwater ecosystems',
 'journal of the american society for horticultural science': 'journal of the american society for horticultural science',
 'microbial ecology': 'microbial ecology',
 'fisheries research': 'fisheries research',
 'oikos': 'oikos',
 'laboratory animals': 'laboratory animals',
 'peanut science': 'peanut science',
 'biodiversity and conservation': 'biodiversity and conservation',
 'biology and fertility of soils': 'biology and fertility of soils',
 'aquaculture international': 'aquaculture international',
 'annual review of entomology': 'annual review of entomology',
 'reproduction, fertility and development': 'reproduction, fertility and development',
 'journal of forestry research': 'journal of forestry research',
 'journal of soils and sediments': 'journal of soils and sediments',
 'crustaceana': 'crustaceana',
 'organic agriculture': 'organic agriculture',
 'plant health

In [160]:
col_name = "journal_name"
db = SQLiteDB()
db.initialise()
cnt = 0
for ind, row in data.iterrows():
    name = row[9].lower().strip() if row[9] is not None else None
    db._update_col(ind, col_name, name_to_standard[name])
db.terminate()

In [161]:
all_journals = set(name_to_standard.values())

In [165]:
import json
with open("all_queryable_journals.json", "w") as f:
    json.dump(list(all_journals), f)

In [168]:
sum(pd.isna(data[14]))

1